In [1]:
import numpy as np
import onnx
from onnx.helper import(
    make_node,make_graph,make_model,make_tensor_value_info
)
from onnx.numpy_helper import from_array
from onnx.checker import check_model
from onnxruntime import InferenceSession

from_array:将数组转为ONNX的Tensor

In [2]:
value = np.array([0],dtype=np.float32)
zero = from_array(value,name='zero')

创建tensor张量


In [3]:
X = make_tensor_value_info('X',onnx.TensorProto.FLOAT,[None,None])
Y = make_tensor_value_info('Y',onnx.TensorProto.FLOAT,[None,None])

make_node:创建算子,rsum求和，cond为布尔值，用于输出和0比较的大小的结果

In [4]:
rsum = make_node('ReduceSum',['X'],['rsum'])
cond = make_node('Greater',['rsum','zero'],['cond'])

创建一个张量：make_tensor_value_info(name, elem_type, shape)


In [5]:
then_out = make_tensor_value_info('then_out',onnx.TensorProto.FLOAT,None)
then_cst = from_array(np.array([1]).astype(np.float32))

定义一个常量节点，无输入，但有输出'then_out'

In [6]:
then_const_node = make_node(
    'Constant', inputs=[],
    outputs=['then_out'],
    value=then_cst, name='cst1')

make_graph(  
    nodes,                 # 节点列表，包含了计算图中的所有节点  
    name,                  # 计算图的名称  
    inputs,                # 输入张量的列表  
    outputs,               # 输出张量的列表  
    initializer=None,      # （可选）初始化张量的列表，通常是常量张量  
    value_info=None,       # （可选）张量的描述信息列表，用于定义输入输出张量的详细信息  
    doc_string=None        # （可选）图的描述字符串  
)  

In [7]:
then_body = make_graph(
    [then_const_node],'then_body',[],[then_out]
)

In [8]:
else_out = make_tensor_value_info(
    "else_out",onnx.TensorProto.FLOAT, [5])
else_cst = from_array(np.array([-1]).astype(np.float32))

else_const_node = make_node(
    'Constant',inputs=[],
    outputs=['else_out'],
    value=else_cst,name='cst2')

else_body = make_graph(
    [else_const_node],'else_body',
    [],[else_out])

onnx.helper.make_node(
    op_type,                # 运算类型（算子名称）
    inputs,                 # 输入张量的名称列表
    outputs,                # 输出张量的名称列表
    name=None,              # 节点的名称（可选）
    domain=None,            # 计算域（可选），通常用于自定义算子
    attributes=None,        # 节点的属性（可选）
    op_version=None,        # 操作版本（可选）
    doc_string=None         # 节点的文档描述（可选）
)


In [9]:
if_node = onnx.helper.make_node(
    'If',['cond'],['Y'],
    then_branch=then_body,
    else_branch=else_body)

In [10]:
graph = make_graph([rsum,cond,if_node],'if',[X],[Y],[zero])
onnx_model = make_model(graph)
check_model(onnx_model)

In [11]:
with open("onnx_if_sign.onnx","wb") as f:
    f.write(onnx_model.SerializeToString())
    
sess = InferenceSession(onnx_model.SerializePartialToString(),providers=['CUDAExecutionProvider'])

x = np.ones((3,2),dtype=np.float32)
res = sess.run(None,{'X':x})

print("result", res)
print()

print(onnx_model)

/root/anaconda3/envs/onnx/lib/python3.12/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


result [array([1.], dtype=float32)]

ir_version: 10
graph {
  node {
    input: "X"
    output: "rsum"
    op_type: "ReduceSum"
  }
  node {
    input: "rsum"
    input: "zero"
    output: "cond"
    op_type: "Greater"
  }
  node {
    input: "cond"
    output: "Y"
    op_type: "If"
    attribute {
      name: "else_branch"
      g {
        node {
          output: "else_out"
          name: "cst2"
          op_type: "Constant"
          attribute {
            name: "value"
            t {
              dims: 1
              data_type: 1
              raw_data: "\000\000\200\277"
            }
            type: TENSOR
          }
        }
        name: "else_body"
        output {
          name: "else_out"
          type {
            tensor_type {
              elem_type: 1
              shape {
                dim {
                  dim_value: 5
                }
              }
            }
          }
        }
      }
      type: GRAPH
    }
    attribute {
      name: "then

2025-03-31 11:41:24.506028429 [W:onnxruntime:, graph.cc:113 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1} target:{-1,-1}. Falling back to lenient merge.
